In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150, # 메시지 토큰 수의 최대값 , 메시지 요약되기 limit
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

In [2]:
add_message("탑건","🛩️👨‍✈️🔥")
add_message("대부","👨‍👨‍👦🔫🍝")

In [3]:
get_history()

{'history': [HumanMessage(content='탑건'),
  AIMessage(content='🛩️👨\u200d✈️🔥'),
  HumanMessage(content='대부'),
  AIMessage(content='👨\u200d👨\u200d👦🔫🍝')]}

In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 제목을 받으면 3글자의 이모티콘으로 대답해주는 ai 야!"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm 

def invoke_chain(question):
    result = chain.invoke({"question": question})
    print(result)
    memory.save_context({"input": question}, {"output": result.content})


In [11]:
invoke_chain("기생충")

content='🐜🏠💰'


In [13]:
load_memory({})

[HumanMessage(content='탑건'),
 AIMessage(content='🛩️👨\u200d✈️🔥'),
 HumanMessage(content='대부'),
 AIMessage(content='👨\u200d👨\u200d👦🔫🍝'),
 HumanMessage(content='기생충'),
 AIMessage(content='🐜🏠💰')]